In [65]:
import pandas as pd
import seaborn as sns 
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy.stats import sem

%matplotlib inline
%config Completer.use_jedi = False


#import required sklearn for logistic regression classifier

import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix,roc_auc_score, f1_score
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, RepeatedStratifiedKFold, RepeatedKFold, cross_validate
from sklearn.feature_selection import mutual_info_classif,VarianceThreshold, SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [66]:
#Load the filtered isoform switch file and the file containing all genes for COAD
#Only matched samples / files used

isoform_switch_file = '/Users/jake/OneDrive - University of Glasgow/Project/COAD/isoform_switch/COAD_FPKM_protein_isoform_filtered_matched.csv'
all_genes_file = '/Users/jake/OneDrive - University of Glasgow/Project/COAD/GeneExpression_MAF/COAD_FPKM_protein_all_matched.csv'

isoform_switch = pd.read_csv(isoform_switch_file,header=[0,1],sep='\t',index_col=[0])
all_genes = pd.read_csv(all_genes_file,header=[0,1],sep='\t',index_col=[0])


In [67]:
isoform_switch

Gene_symbol,CTTN,FBLN2,FLNA,ARHGEF9,SLC39A14,ATP6V1C2,ISLR,MYH11,SRI,UGP2,IL1R2,HKDC1,CD44,Cancer
Gene_ensembl_id,ENSG00000085733,ENSG00000163520,ENSG00000196924,ENSG00000131089,ENSG00000104635,ENSG00000143882,ENSG00000129009,ENSG00000133392,ENSG00000075142,ENSG00000169764,ENSG00000115590,ENSG00000156510,ENSG00000026508,Cancer
TCGA-AA-3522,34.492193,1.847002,19.812343,3.636078,26.804372,0.583120,7.840883,1.812990,68.843328,30.104854,4.492162,14.452351,71.507735,1
TCGA-AA-3517,59.752870,6.863174,47.027555,4.663242,37.329810,3.091385,12.402785,4.193490,39.202101,10.563102,0.615541,67.231999,55.296634,1
TCGA-A6-2686,20.172616,11.047990,39.440702,2.686910,29.445689,1.358366,26.509775,4.201186,93.478642,12.939356,3.460002,4.743835,66.239432,1
TCGA-AA-3518,43.747229,3.768492,37.933336,2.250100,18.019370,2.310897,9.155425,0.740950,16.350869,14.671055,2.468007,16.825744,54.122628,1
TCGA-AZ-6599,32.496824,1.224267,9.737549,2.394307,24.810476,0.616765,1.208672,3.877568,49.209436,14.396411,5.636202,6.163669,60.051651,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-A6-2682,36.195520,9.756399,40.254574,5.116727,27.502997,0.668204,7.510906,17.512665,116.629904,57.037639,10.666862,11.973689,12.996430,0
TCGA-AA-3522,38.606857,16.125248,45.252554,5.323429,31.354408,0.477939,10.783217,24.119451,153.386318,74.743021,18.051094,9.957414,11.233170,0
TCGA-AA-3662,31.916105,21.770961,92.316274,6.593554,28.567929,1.115913,16.008965,125.767640,69.142586,45.311255,49.908591,2.679323,15.903117,0


In [68]:
#Check isoform_switch for and missing data

isoform_switch.isnull().sum().unique()

array([0])

In [69]:
#Check all genes file for missing data

all_genes.isnull().sum().sort_values(ascending = False)

#AFT7-NPFF is 'an important paralog of the gene AFT7' => could combine data for both??

Gene_symbol  Gene_ensembl_id
ATF7         ENSG00000267281    46
ATF7-NPFF    ENSG00000267281    41
A1BG         ENSG00000121410     0
PPP1R9A      ENSG00000158528     0
PPP2R1B      ENSG00000137713     0
                                ..
GBP5         ENSG00000154451     0
GBP4         ENSG00000162654     0
GBP3         ENSG00000117226     0
GBP2         ENSG00000162645     0
ZZEF1        ENSG00000074755     0
Length: 19538, dtype: int64

In [70]:
#For now, will drop the missing data

all_genes.dropna(axis=1,inplace=True)

In [71]:
all_genes.isnull().sum().unique()

array([0])

In [72]:
#Check tumour weight / ratio ... since almost equal will not use 'class_weight' in model

tumour_ratio = sum(isoform_switch['Cancer','Cancer']==1) / sum(isoform_switch['Cancer','Cancer']==0)
tumour_ratio

1.1219512195121952

In [73]:
#Do an initial train test split to extract an 'unseen' test set of data (30%) for both isoform and all genes
#Set stratify as y to keep same ratio in test data

X_full_iso = isoform_switch.drop('Cancer',axis=1,level=0)
y_full_iso = isoform_switch['Cancer','Cancer']

X_develop_iso, X_test_iso, y_develop_iso, y_test_iso = train_test_split(X_full_iso, y_full_iso, test_size=0.3, random_state=1,stratify=y_full_iso)


In [89]:
#Repeat for all genes .. using random sample of same number of genes as isoform

X_full_all = all_genes.drop('Cancer',axis=1,level=0)
y_full_all = all_genes['Cancer','Cancer']

#Take a random sample of genes .. use X_full_iso.columns to acquire same num of genes as isoform
X_random = X_full_all.sample(n=len(X_full_iso.columns),axis=1)

X_develop_ran, X_test_ran, y_develop_ran, y_test_ran = train_test_split(X_random, y_full_all, test_size=0.3, random_state=1,stratify=y_full_all)


In [75]:
#First do simple Logistic Regression with gridsearch to determine optimal parameters

def grid_search_LR(X,y):
    '''Perform logistic regression grid search. Supply development X and y data.'''

    #Logistic Regression with dual = False since n_samples > n_features
    classifier = LogisticRegression(dual=False)

    #Pipeline containing Log Reg classifier and Log Reg with deafault settings for feature engineering
    #Also added scaler as error related to iterations / convergance appears, which can be due to 'poor scaling'
    LR_pipeline = Pipeline(steps=[('scale', StandardScaler()),
                                  ('feature_selection', SelectFromModel(LogisticRegression(dual=False))),
                                  ('classifier',classifier)])


    #Two dictionaries of potential potential parameters to account for penalties and solvers that can/can't go togther
    param_grid = [
        {'classifier__penalty': ['l1'], 'classifier__solver': ['liblinear', 'saga'],
         'classifier__C':[0.01,0.1,1,10,100]},
        {'classifier__penalty': ['l2'], 'classifier__solver': ['liblinear', 'sag', 'saga','newton-cg'],'classifier__C':[0.01,0.1,1,10,100]}
    ]


    #Use repeated stratified k fold to maintian ratio between tumor and normal and do multliple splits of data
    r_stratkfold = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=0)

    grid_search = GridSearchCV(LR_pipeline, param_grid=param_grid, scoring="roc_auc",cv=r_stratkfold)

    #Fit on development data
    grid_search.fit(X, y)
    
    #Create dictinoary to store the best parameters
    best_params = {}
    
    for k,v in grid_search.best_params_.items():
        
        best_params[k] = v
 
    #Return the best_params dictinoary => run this fuction and save as new variable in relation to what X and y run
    return best_params

In [76]:
#Run the gridsearch for both the isoform and all genes development data to create dictionaries to plug into CV

best_params_iso = grid_search_LR(X_develop_iso,y_develop_iso)
best_params_iso

/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarni

/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarni

{'classifier__C': 100,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [77]:
best_params_ran = grid_search_LR(X_develop_ran,y_develop_ran)
best_params_ran

/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarni

/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarni

{'classifier__C': 1, 'classifier__penalty': 'l1', 'classifier__solver': 'saga'}

In [78]:
#Now define model functions that will return cross_validate to evalaute model overall

In [91]:
# Create a function that cross validates the data

def LR_model_cv(X, y, best_params_dict, repeats = 3, splits = 3):
    '''LR model function. Takes in X and y development data, repeats, splits and the required best_params dictinoary'''

    #First perform RepeatedStratifiedKFold
    r_stratkfold = RepeatedStratifiedKFold(n_splits=splits, n_repeats=repeats)
    
    # Create classifier with LR model using best_params_iso from gridsearch
    
    classifier = LogisticRegression(C=best_params_dict['classifier__C'],
                                   penalty= best_params_dict['classifier__penalty'],
                                   solver = best_params_dict['classifier__solver'], 
                                   random_state=1,
                                   dual = False)
    
    #Create pipleine with scaling, feature selection and the classifier
    LR_pipeline = Pipeline(steps=[('scale', StandardScaler()),
                                   ('feature_selection', SelectFromModel(LogisticRegression(dual=False))),
                                   ('classifier',classifier)])
    
    
    # evaluate the cross_validate
    scores_cv = cross_validate(LR_pipeline, X, y, cv=r_stratkfold,
                            scoring=('f1_weighted','roc_auc','balanced_accuracy','precision_weighted','recall_weighted'))
    
    #Define performance metrics to return 
    F1 = str(f"Mean F1 weighted = {scores_cv['test_f1_weighted'].mean():.3f} SEM = {sem(scores_cv['test_f1_weighted']):.3f}")
    ROCAUC = str(f"Mean ROCAUC score = {scores_cv['test_roc_auc'].mean():.3f} SEM = {sem(scores_cv['test_roc_auc']):.3f} ")
    Accuracy = str(f"Mean balanced accuracy score = {scores_cv['test_balanced_accuracy'].mean():.3f} SEM = {sem(scores_cv['test_balanced_accuracy']):.3f} ")
    Precision = str(f"Mean weighted precision = {scores_cv['test_precision_weighted'].mean():.3f} SEM = {sem(scores_cv['test_precision_weighted']):.3f} ")
    Recall = str(f"Mean weighted recall = {scores_cv['test_recall_weighted'].mean():.3f} SEM = {sem(scores_cv['test_recall_weighted']):.3f} ")
    
    return F1, ROCAUC, Accuracy, Precision, Recall

In [92]:
#Compare the two cross validations with 3 repeats and 3 splits

LR_model_cv(X_develop_iso,y_develop_iso,best_params_iso)

/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarni

('Mean F1 weighted = 0.989 SEM = 0.007',
 'Mean ROCAUC score = 1.000 SEM = 0.000 ',
 'Mean balanced accuracy score = 0.988 SEM = 0.008 ',
 'Mean weighted precision = 0.990 SEM = 0.007 ',
 'Mean weighted recall = 0.989 SEM = 0.007 ')

In [93]:
LR_model_cv(X_develop_ran,y_develop_ran,best_params_ran)

/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarni

('Mean F1 weighted = 0.917 SEM = 0.017',
 'Mean ROCAUC score = 0.966 SEM = 0.011 ',
 'Mean balanced accuracy score = 0.919 SEM = 0.017 ',
 'Mean weighted precision = 0.921 SEM = 0.016 ',
 'Mean weighted recall = 0.917 SEM = 0.017 ')

In [83]:
#Now create models utilising the above parameters to perform a test on the unseeen test data

In [94]:
# Create function to perform test

def LR_model_test(X_develop,y_develop,X_test,y_test, best_params_dict):
    '''Function fits the predetermined LR_pipeline for the data before predicting on the unseen test data'''
    
    classifier = LogisticRegression(C=best_params_dict['classifier__C'],
                                   penalty= best_params_dict['classifier__penalty'],
                                   solver = best_params_dict['classifier__solver'], 
                                   random_state=1,
                                   dual = False)
    
    LR_pipeline = Pipeline(steps=[('scale', StandardScaler()),
                                   ('feature_selection', SelectFromModel(LogisticRegression(dual=False))),
                                   ('classifier',classifier)])
    
    #Fit on the develop data
    LR_pipeline.fit(X_develop,y_develop)
    
    #Then make predictions on the unseen test data from the initial train test split
    predictions = LR_pipeline.predict(X_test)
    
    #Return classification report and ROCAUC score
    print (classification_report(predictions,y_test,target_names=['Normal - 0','Cancer - 1']))
    ROCAUC =  str(f'ROCAUC score: {roc_auc_score(predictions,y_test):.3f}')
    
    return ROCAUC

In [86]:
#Now run on both set of data to compare the results

In [95]:
LR_model_test(X_develop_iso,y_develop_iso,X_test_iso,y_test_iso,best_params_iso)

              precision    recall  f1-score   support

  Normal - 0       1.00      1.00      1.00        13
  Cancer - 1       1.00      1.00      1.00        14

    accuracy                           1.00        27
   macro avg       1.00      1.00      1.00        27
weighted avg       1.00      1.00      1.00        27



/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


'ROCAUC score: 1.000'

In [96]:
LR_model_test(X_develop_ran,y_develop_ran,X_test_ran,y_test_ran,best_params_ran)

              precision    recall  f1-score   support

  Normal - 0       0.92      0.92      0.92        13
  Cancer - 1       0.93      0.93      0.93        14

    accuracy                           0.93        27
   macro avg       0.93      0.93      0.93        27
weighted avg       0.93      0.93      0.93        27



/Users/jake/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


'ROCAUC score: 0.926'